### **主題：啤酒評論評分預測 - 分詞器(Tokenizer)**
### **說明：**
繼續上次啤酒評鑑資料集的練習，還記得在上次的資料初步分析之後，對於 BERT 模型最大長度的限制是 256 嗎？<br />
然而這還只是未使用分詞器所得到的結論，當真正使用分詞器之後，每評論句的token數可能會與先前評估有所不同，因此這次作業主要以BERT分詞器來驗證上次作業得到的結論是否需要修正。
### **題目：**
1. 創建英文 BERT 所使用的分詞器，提供下一題分析以及後續訓練使用
2. 以分詞器生成每個評論語句的token，並評估上次得到的最大長度限制是否合理
#### **提示: 不要忘記加上[SEP]與[CLS]**

In [1]:
import os
import pandas as pd
from transformers import BertTokenizer

In [2]:
TRAIN = pd.read_json(os.path.join('data', 'train_set.json'), encoding='utf-8')
TEST = pd.read_json(os.path.join('data', 'test_set.json'), encoding='utf-8')

In [3]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
TOKENIZER = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [4]:
text = TRAIN['review/text'][0][:250]

tokens = TOKENIZER.tokenize(text)
token_ids = TOKENIZER.convert_tokens_to_ids(tokens)

print(tokens[:50])
print(token_ids[:50])

['Po', '##urs', 'a', 'cloud', '##ed', 'gold', 'with', 'a', 'thin', 'white', 'head', '.', 'No', '##se', 'is', 'quite', 'floral', 'with', 'a', 'larger', 'amount', 'of', 'spices', 'added', '.', 'De', '##fin', '##ite', '##ly', 'a', 's', '##pice', 'forward', 'f', '##rag', '##rance', '.', 'F', '##lav', '##or', 'has', 'an', 'odd', 'burn', 'that', 'hits', 'on', 'the', 'first', 'sip']
[18959, 7719, 170, 7180, 1174, 2284, 1114, 170, 4240, 1653, 1246, 119, 1302, 2217, 1110, 2385, 22504, 1114, 170, 2610, 2971, 1104, 25133, 1896, 119, 3177, 16598, 3150, 1193, 170, 188, 15633, 1977, 175, 20484, 10555, 119, 143, 9516, 1766, 1144, 1126, 5849, 6790, 1115, 4919, 1113, 1103, 1148, 11456]


In [5]:
MAX_SEQ_LEN = 512

In [6]:
def get_encoding(text, max_seq_len=MAX_SEQ_LEN):
    encoding = TOKENIZER.encode_plus(
        text,
        max_length=max_seq_len,
        truncation=True,
        add_special_tokens=True,
        return_token_type_ids=False,
        padding=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    return [t for t in encoding['input_ids'][0] if t != 0]

In [7]:
TRAIN['token_number'] = TRAIN['review/text'].apply(str).apply(get_encoding).apply(len)

In [8]:
TRAIN['token_number'].describe([0.8, 0.9, 0.95])

count    45000.000000
mean       181.997133
std         90.060288
min          3.000000
50%        160.000000
80%        242.000000
90%        302.000000
95%        364.000000
max        512.000000
Name: token_number, dtype: float64

### 結論
Tokenize 後可以發現語句的 Token 數超出先前使用空白分割的 256 (因為 BERT 使用 word piece)